In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [3]:
words = open('names.txt',"r").read().splitlines()
len(words)

32033

In [6]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()} 
len(stoi)

27

In [16]:
#build the dataset
block_size = 3 # n-grams.
X, Y = [],[]

for w in words[:6]:
    print(w)

    context = [0] * block_size
    print(f"New context = {context} ")
    for ch in w + '.':
        ix = stoi[ch]
        X.append(context)
        Y.append(ix)
        print(''.join(itos[i] for i in context), '--->', itos[ix])
        context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)


yuheng
New context = [0, 0, 0] 
... ---> y
..y ---> u
.yu ---> h
yuh ---> e
uhe ---> n
hen ---> g
eng ---> .
diondre
New context = [0, 0, 0] 
... ---> d
..d ---> i
.di ---> o
dio ---> n
ion ---> d
ond ---> r
ndr ---> e
dre ---> .
xavien
New context = [0, 0, 0] 
... ---> x
..x ---> a
.xa ---> v
xav ---> i
avi ---> e
vie ---> n
ien ---> .
jori
New context = [0, 0, 0] 
... ---> j
..j ---> o
.jo ---> r
jor ---> i
ori ---> .
juanluis
New context = [0, 0, 0] 
... ---> j
..j ---> u
.ju ---> a
jua ---> n
uan ---> l
anl ---> u
nlu ---> i
lui ---> s
uis ---> .
erandi
New context = [0, 0, 0] 
... ---> e
..e ---> r
.er ---> a
era ---> n
ran ---> d
and ---> i
ndi ---> .


In [17]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([43, 3]), torch.int64, torch.Size([43]), torch.int64)

In [18]:
# build the dataset
block_size = 3 # context length: how many characters do we take to predict the next one?

def build_dataset(words):  
  X, Y = [], []
  for w in words:

    #print(w)
    context = [0] * block_size
    for ch in w + '.':
      ix = stoi[ch]
      X.append(context)
      Y.append(ix)
      #print(''.join(itos[i] for i in context), '--->', itos[ix])
      context = context[1:] + [ix] # crop and append

  X = torch.tensor(X)
  Y = torch.tensor(Y)
  print(X.shape, Y.shape)
  return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte, Yte = build_dataset(words[n2:])

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [19]:
C = torch.randn((27, 2)) # Embedding 27 characters into 2 dimensional vector space.

In [20]:
emb = C[X] # Intelligent index access from torch.
emb.shape # Batch size, number of input words in context, embedding dimension.

torch.Size([43, 3, 2])

In [21]:
W1 = torch.randn((6, 100))
b1 = torch.randn(100)

In [22]:
h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
h

tensor([[ 0.9290,  1.0000,  0.9992,  ...,  0.5289,  1.0000, -0.9997],
        [-0.9671,  0.9952,  0.8684,  ..., -0.9434,  0.9997, -1.0000],
        [-0.9999, -1.0000, -0.9914,  ..., -1.0000, -0.1219, -1.0000],
        ...,
        [ 0.9998,  0.6993,  0.9911,  ...,  0.9697, -0.3398,  0.9946],
        [-0.8935, -0.9990, -0.7509,  ..., -0.4154, -0.2982,  0.8595],
        [-1.0000, -0.9999, -1.0000,  ..., -0.9990, -0.9415, -0.9034]])